# Modelo de Clasificación para la Selección Óptima de Planes Móviles en Megaline

El objetivo de este proyecto es ayudar a Megaline a optimizar la recomendación de sus nuevos planes móviles, Smart o Ultra. Actualmente, muchos clientes siguen utilizando planes heredados, y queremos asegurarnos de que obtengan el plan que mejor se ajuste a su uso real.

Para lograr esto, vamos a desarrollar un modelo de clasificación que analizará el comportamiento de los suscriptores y recomendará uno de los nuevos planes de manera precisa. Ya hemos procesado los datos y estamos listos para crear el modelo, priorizando una exactitud mínima del 75%. Utilizaremos el conjunto de datos disponible para verificar y garantizar que el modelo cumple con los estándares de calidad esperados.

Este enfoque ayudará a Megaline a ofrecer a sus clientes planes más adecuados, optimizando tanto la satisfacción del cliente como el uso de los recursos de la empresa.

<span style='color:#20B2AA'> Preparación y Segmentación de Datos para el Modelo de Clasificación </span>

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

In [4]:
# Intentar cargar el archivo CSV
file_path = 'users_behavior.csv'
data = pd.read_csv(file_path)

# Mostrar las primeras filas del dataset
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<span style='color:#800080'> Explicación: </span> Se importó y cargó el conjunto de datos, y se realizó un examen de las primeras filas para obtener una vista preliminar de la estructura y contenido del dataset.

In [5]:
# Dividir el dataset en conjunto de entrenamiento (70%) y conjunto de validación/prueba (30%)
train_data, temp_data = train_test_split(data, test_size=0.3, random_state=42)

# Dividir el conjunto de validación/prueba en conjunto de validación (15%) y conjunto de prueba (15%)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Mostrar el tamaño de los conjuntos
print(f'Tamaño del conjunto de entrenamiento: {train_data.shape}')
print(f'Tamaño del conjunto de validación: {val_data.shape}')
print(f'Tamaño del conjunto de prueba: {test_data.shape}')

Tamaño del conjunto de entrenamiento: (2249, 5)
Tamaño del conjunto de validación: (482, 5)
Tamaño del conjunto de prueba: (483, 5)


<span style='color:#800080'> Explicación: </span> Dividir los datos: Usaremos train_test_split dos veces: una vez para dividir entre el conjunto de entrenamiento y el conjunto de validación/prueba, y luego para dividir el conjunto de validación/prueba.

<span style='color:#20B2AA'> Investigar la calidad de diferentes modelos cambiando los hiperparámetros </span>

In [6]:
# Definir los modelos a probar
models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'KNN': KNeighborsClassifier()
}

# Hiperparámetros a probar para cada modelo
param_grids = {
    'Logistic Regression': {'C': [0.1, 1, 10]},
    'Random Forest': {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]},
    'KNN': {'n_neighbors': [3, 5, 7]}
}

# Resultados almacenados
best_models = {}
for model_name, model in models.items():
    grid_search = GridSearchCV(model, param_grids[model_name], cv=5, scoring='accuracy')
    grid_search.fit(train_data.drop('is_ultra', axis=1), train_data['is_ultra'])  # Usamos 'is_ultra' como columna objetivo
    best_models[model_name] = grid_search.best_estimator_
    print(f"Mejor modelo para {model_name}: {grid_search.best_params_}")

# Evaluar los mejores modelos
for model_name, model in best_models.items():
    val_predictions = model.predict(val_data.drop('is_ultra', axis=1))
    accuracy = accuracy_score(val_data['is_ultra'], val_predictions)
    print(f"Accuracy en conjunto de validación para {model_name}: {accuracy}")

Mejor modelo para Logistic Regression: {'C': 0.1}
Mejor modelo para Random Forest: {'max_depth': 10, 'n_estimators': 200}
Mejor modelo para KNN: {'n_neighbors': 7}
Accuracy en conjunto de validación para Logistic Regression: 0.7634854771784232
Accuracy en conjunto de validación para Random Forest: 0.8236514522821576
Accuracy en conjunto de validación para KNN: 0.7987551867219918


<span style='color:#800080'> Explicación: </span> GridSearchCV: Esta función busca automáticamente la mejor combinación de hiperparámetros, realizando una validación cruzada en cada combinación.
Entrenamiento de diferentes modelos: Probamos Logistic Regression, Random Forest y KNN, variando los hiperparámetros como C para la regresión logística, el número de estimadores para el bosque aleatorio y el número de vecinos en KNN.
Mejor modelo: Se obtiene el mejor modelo para cada técnica y se reporta su rendimiento en el conjunto de validación.
Métrica de calidad: Utilizamos la precisión como métrica de rendimiento.

<span style='color:#20B2AA'> Comprobar la calidad del modelo usando el conjunto de prueba </span>

In [7]:
# Seleccionamos el mejor modelo, por ejemplo el Random Forest
best_model = best_models['Random Forest']  # Cambia este modelo si otro ha sido mejor

# Predecir en el conjunto de prueba
test_predictions = best_model.predict(test_data.drop('is_ultra', axis=1))

# Evaluar la calidad en el conjunto de prueba
test_accuracy = accuracy_score(test_data['is_ultra'], test_predictions)

print(f"Precisión del mejor modelo en el conjunto de prueba: {test_accuracy}")

Precisión del mejor modelo en el conjunto de prueba: 0.782608695652174


<span style='color:#800080'> Explicación: </span> Columna objetivo: Hemos utilizado la columna is_ultra como la columna que queremos predecir.
Modelos: Usamos Logistic Regression, Random Forest y KNN y ajustamos sus hiperparámetros utilizando GridSearchCV.
Evaluación: Comprobamos la precisión en el conjunto de validación y, finalmente, en el conjunto de prueba para determinar la calidad final del mejor modelo.

<span style='color:#008000'> ADICIONAL: Código para la prueba de cordura </span>

<span style='color:#D2691E'> 1. Modelo Baseline (Regla trivial): </span>
Este modelo simplemente predice siempre la clase mayoritaria (la que aparece con mayor frecuencia).

In [8]:
# Crear un modelo de regla trivial (siempre predice la clase mayoritaria)
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(train_data.drop('is_ultra', axis=1), train_data['is_ultra'])

# Evaluar el modelo trivial en el conjunto de validación
dummy_val_predictions = dummy_clf.predict(val_data.drop('is_ultra', axis=1))
dummy_accuracy = accuracy_score(val_data['is_ultra'], dummy_val_predictions)

print(f"Precisión del modelo trivial (baseline) en el conjunto de validación: {dummy_accuracy}")

Precisión del modelo trivial (baseline) en el conjunto de validación: 0.7074688796680498


<span style='color:#D2691E'>2. Aleatorización de etiquetas: </span>
Este paso aleatoriza las etiquetas para ver si el modelo puede ajustarse a ruido puro. Debería obtener resultados similares a los de una predicción aleatoria si el modelo no está "aprendiendo" patrones reales.

In [9]:
# Aleatorizar las etiquetas
random_labels = np.random.permutation(train_data['is_ultra'])

# Entrenar el modelo con etiquetas aleatorias
model_with_random_labels = RandomForestClassifier()
model_with_random_labels.fit(train_data.drop('is_ultra', axis=1), random_labels)

# Evaluar el modelo en el conjunto de validación
random_val_predictions = model_with_random_labels.predict(val_data.drop('is_ultra', axis=1))
random_accuracy = accuracy_score(val_data['is_ultra'], random_val_predictions)

print(f"Precisión del modelo entrenado con etiquetas aleatorias: {random_accuracy}")

Precisión del modelo entrenado con etiquetas aleatorias: 0.7012448132780082


<span style='color:#D2691E'> 3. Comparación con el modelo real: </span>
Finalmente, comparamos los resultados del modelo con etiquetas reales frente al modelo con etiquetas aleatorias.

In [10]:
# Evaluar el modelo real (con etiquetas reales)
real_val_predictions = best_model.predict(val_data.drop('is_ultra', axis=1))
real_accuracy = accuracy_score(val_data['is_ultra'], real_val_predictions)

print(f"Precisión del mejor modelo en el conjunto de validación (con etiquetas reales): {real_accuracy}")

# Comparar con las etiquetas aleatorias
if real_accuracy > random_accuracy:
    print("El modelo con etiquetas reales tiene un mejor rendimiento, lo que sugiere que el modelo no está ajustando ruido.")
else:
    print("El modelo no mejora con respecto a etiquetas aleatorias, podría estar ajustando ruido o datos superficiales.")

Precisión del mejor modelo en el conjunto de validación (con etiquetas reales): 0.8236514522821576
El modelo con etiquetas reales tiene un mejor rendimiento, lo que sugiere que el modelo no está ajustando ruido.


<span style='color:#800080'> Explicación: </span>
Modelo Baseline: Este modelo siempre predice la clase más frecuente en los datos de entrenamiento. Si tu modelo no es capaz de superar esta predicción trivial, algo está mal.

Aleatorización de etiquetas: Entrenamos el modelo con etiquetas aleatorias para ver si puede ajustar patrones reales en lugar de ruido. Si el modelo real no tiene un mejor rendimiento que el modelo aleatorizado, significa que no está aprendiendo patrones significativos.

Comparación: Comparamos la precisión de los modelos con etiquetas reales y aleatorias. El objetivo es que el modelo con etiquetas reales tenga una precisión significativamente mayor.

<span style='color:#0000FF'> Conclusión del análisis y pruebas realizadas: </span>

El análisis realizado mostró que el modelo Random Forest fue el más efectivo para predecir si un usuario tiene el plan Ultra o Smart. Ajustamos los hiperparámetros utilizando GridSearchCV y logramos una precisión alta tanto en el conjunto de validación como en el conjunto de prueba.

La prueba de cordura demostró que el modelo está aprendiendo patrones reales en los datos, ya que superó tanto al modelo trivial como al entrenado con etiquetas aleatorias. Esto confirma que el modelo no está ajustando ruido, sino que es capaz de generalizar bien.

En resumen, el Random Forest es un modelo sólido y confiable para este problema.